## Настройки Colab

In [ ]:
# Настройка пользователя (сделать один раз)
!git config --global user.email "nabludatellip@gmail.com"
!git config --global user.name "ProninPV"

In [ ]:
!git clone https://github.com/ProninPV/ml-regression_concrete-strength.git
%cd ml-regression_concrete-strength

In [ ]:
!pip install catboost

## 6.0 Импорты библиотек

In [21]:
import os
import yaml
import logging
import pickle
import numpy as np
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import levene
from scipy.stats import ttest_ind
from typing import List, Any, Optional, Tuple, Dict, Union
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score, KFold, RepeatedKFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import time
import psutil
from tqdm import tqdm
import gc
import optuna

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [5]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [6]:
# В Colab проект клонируется в /content/
# Устанавливаем правильную рабочую директорию
# project_root = Path('/content/ml-regression_concrete-strength')

# Определяем корень проекта
cwd = Path().resolve()
project_root = cwd.parent

# Добавляем корень проекта в sys.path (этого достаточно)
sys.path.append(str(project_root))

# Проверяем наличие конфиг файла
config_path = project_root / "config" / "config.yaml"
print(f"Looking for config at: {config_path}")

# Загрузка данных из config.yaml
from src.data import downloader, loader, preprocessor, saving
from src.features import feat_preprocessing
from src.modeling import modeling

# Передаем путь явно
config = loader.load_config(config_path)
print("✅ Config loaded successfully!")

Looking for config at: D:\Skills\Kaggle\ml-regression_concrete-strength\config\config.yaml
✅ Config loaded successfully!


## 6.1. Загрузка данных

In [7]:
# Загрузка train
df_train = loader.data_load_preprocessed(data_type='train',
                                         config=config)

[⧗] Загружаю данные из: D:\Skills\Kaggle\ml-regression_concrete-strength\data\processed\eda_data_train.pkl
[✓] Данные успешно загружены. Форма: (781, 11)


In [8]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40,0.768743,0.032923


In [9]:
# Загрузка отчета по экспериментам
modeling_result = modeling.load_sorted_modeling_report(config)
modeling_result

,outlier_strategy,model_name,model_type,dataset_size,mean_rmse,std_rmse,training_time_sec,memory_used_mb
0,['abnormal'],CatBoostRegressor,trees_models,770,4.2595,0.3446,13.49,0.44
1,['abnormal'],CatBoostRegressor,linear_models,770,4.2608,0.3868,11.55,12.38
2,"['gost_binar', 'combine']",CatBoostRegressor,linear_models,770,4.3196,0.4173,13.77,0.21
3,"['gost_binar', 'combine']",CatBoostRegressor,trees_models,770,4.3260,0.3906,14.72,0.03
4,['combine'],CatBoostRegressor,linear_models,770,4.3304,0.4210,11.68,0.15
...,...,...,...,...,...,...,...,...
105,['abnormal'],ElasticNet,trees_models,770,8.2662,0.4285,1.57,0.00
106,"['gost_binar', 'combine']",ElasticNet,linear_models,770,8.3014,0.4700,1.47,0.00
107,['gost_binar'],ElasticNet,linear_models,781,8.3183,0.5227,1.38,0.00
108,['abnormal'],ElasticNet,linear_models,770,8.3673,0.4635,1.29,0.00


In [10]:
# Определяем лучшую стратию
best_strategy = modeling.get_best_model_strategy(modeling_result)

best_model = best_strategy['model']
best_model_type = best_strategy['model_type']
best_rmse = best_strategy['rmse']
best_outlier_strategy = eval(best_strategy['outlier_strategy'])

In [11]:
best_strategy

{'model': 'CatBoostRegressor',
 'model_type': 'trees_models',
 'outlier_strategy': "['abnormal']",
 'rmse': np.float64(4.2595)}

## 6.2. Предобработка данных

In [12]:
# Разделение на признаки и целевую переменную
X = df_train.drop('Strength', axis=1)
y = df_train['Strength']
y_name = y.name

In [28]:
# Конфигурация для преобразования признаков
feature_config = {
    'trend_settings': {
        'names': ['Linear', 'Log', 'Sqrt']  # Только безопасные
    }
}

In [15]:
# Обрабатываем выбросы для лучшей стратегии 
outlier_handler = feat_preprocessing.OutlierHandler(
    strategies=best_outlier_strategy,
    config=config,
    target_col=y_name
)

X_processed, y_processed = outlier_handler.fit_transform(X, y)

In [17]:
X_processed

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,0.768743,0.032923
...,...,...,...,...,...,...,...,...,...,...
793,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,3,0.619355,0.000000
794,213.5,0.0,174.2,159.2,11.7,1043.6,771.9,100,0.745667,0.054801
795,304.8,0.0,99.6,196.0,9.8,959.4,705.2,28,0.643045,0.032152
797,288.0,192.0,0.0,192.0,0.0,932.0,717.8,28,0.666667,0.000000


## 6.2. Подбор гиперпараметров для CatboostRegressor

#### Определение фнукций

In [18]:
def create_catboost_params(trial, config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Создает параметры для CatBoost на основе конфига Optuna.
    """
    params = {}
    param_space = config['catboost_params']['search_space']
    
    # Динамически создаем параметры из search_space
    for param_name, param_config in param_space.items():
        suggest_method = getattr(trial, param_config['type'])
        
        if param_config['type'] == 'suggest_int':
            params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])
        elif param_config['type'] == 'suggest_float':
            if param_config.get('log', False):
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'], log=True)
            else:
                params[param_name] = suggest_method(param_name, param_config['low'], param_config['high'])
    
    # Добавляем фиксированные параметры
    params.update(config['catboost_params']['fixed_params'])
    
    return params

In [19]:
def create_cv_strategy(config: Dict[str, Any]):
    """
    Создает стратегию кросс-валидации из конфига.
    """
    cv_config = config['cross_validation']
    if cv_config['strategy'] == 'RepeatedKFold':
        from sklearn.model_selection import RepeatedKFold
        return RepeatedKFold(
            n_splits=cv_config['n_splits'],
            n_repeats=cv_config['n_repeats'],
            random_state=cv_config['random_state']
        )
    else:
        from sklearn.model_selection import KFold
        return KFold(n_splits=cv_config['n_splits'], shuffle=True, random_state=cv_config['random_state'])

In [22]:
class ProgressBarCallback:
    """Callback для отображения прогресса Optuna с tqdm."""
    
    def __init__(self, n_trials: int):
        self.n_trials = n_trials
        self.pbar = None
        self.current_trial = 0
        
    def __call__(self, study: optuna.Study, trial: optuna.Trial):
        if self.pbar is None:
            self.pbar = tqdm(total=self.n_trials, desc="Optuna Trials", unit="trial")
        
        self.current_trial += 1
        self.pbar.update(1)
        self.pbar.set_postfix({
            'best_rmse': f"{study.best_value:.4f}",
            'trial': self.current_trial
        })
        
        if self.current_trial >= self.n_trials:
            self.pbar.close()

In [31]:
# Проверяем, что лучшая модель - CatBoost
if best_strategy['model'] != 'CatBoostRegressor':
    print(f"⚠️  Лучшая модель не CatBoost, а {best_strategy['model']}")

print("🎯 Начинаем тюнинг лучшей CatBoost модели:")
print(f"   Модель: {best_strategy['model']}")
print(f"   Стратегия выбросов: {best_strategy['outlier_strategy']}")
print(f"   Базовый RMSE: {best_strategy['rmse']:.4f}")
print("⏳ Запускаем Optuna...")

# Создаем стратегию CV
cv_strategy = create_cv_strategy(config)

# Функция цели для Optuna
def objective(trial):
    # Получаем параметры из конфига
    params = create_catboost_params(trial, config)
    
    # Создаем модель CatBoost с отключенным выводом
    params['verbose'] = False
    model = CatBoostRegressor(**params)
    
    # Создаем полный пайплайн
    pipeline = modeling.create_full_pipeline(
        outlier_strategy=eval(best_strategy['outlier_strategy']),
        model_type=best_strategy['model_type'],
        model=model,
        feature_config=feature_config,
        y_name=y_name
    )
    
    # Оценка с кросс-валидацией
    scores = cross_val_score(
        pipeline, X_processed, y_processed,
        cv=cv_strategy,
        scoring='neg_root_mean_squared_error',
        n_jobs=1,
        error_score='raise'
    )
    
    rmse = -scores.mean()
    
    # Сохраняем дополнительную информацию
    trial.set_user_attr("std_rmse", scores.std())
    trial.set_user_attr("n_features", X_train_raw.shape[1])
    
    return rmse

# Настраиваем параметры запуска
# n_trials = n_trials or optuna_config['optuna']['n_trials']
n_trials = 3

# Создаем study
study = optuna.create_study(
    direction=config['optuna']['direction'],
    study_name=f"catboost_tuning",
    pruner=optuna.pruners.HyperbandPruner()
)

# Создаем прогресс-бар
progress_callback = ProgressBarCallback(n_trials)

# Запускаем оптимизацию
study.optimize(
    objective, 
    n_trials=n_trials,
    callbacks=[progress_callback],
    show_progress_bar=True  # Отключаем встроенный прогресс-бар
)

# Создаем лучшую модель
best_params = study.best_params
best_params['verbose'] = False

best_model = CatBoostRegressor(**best_params)

best_pipeline = create_full_pipeline(
    outlier_strategy=eval(best_strategy['outlier_strategy']),
    model_type=best_strategy['model_type'],
    model=best_model,
    feature_config=feature_config,
    y_name=y_name
)

# Обучаем пайплайн на всех данных
best_pipeline.fit(X_train_raw, y_train)

# Рассчитываем улучшение
improvement = best_strategy['rmse'] - study.best_value
improvement_percent = (improvement / best_strategy['rmse']) * 100

print("\n✅ Тюнинг завершен!")
print(f"📊 Результаты:")
print(f"   До тюнинга: {best_strategy['rmse']:.4f}")
print(f"   После тюнинга: {study.best_value:.4f}")
print(f"   Улучшение: {improvement:.4f} ({improvement_percent:.1f}%)")

[I 2025-11-19 08:47:58,439] A new study created in memory with name: catboost_tuning


🎯 Начинаем тюнинг лучшей CatBoost модели:
   Модель: CatBoostRegressor
   Стратегия выбросов: ['abnormal']
   Базовый RMSE: 4.2595
⏳ Запускаем Optuna...


  0%|                                                                                                                                                        | 0/3 [01:13<?, ?it/s]


[W 2025-11-19 08:49:11,695] Trial 0 failed with parameters: {'iterations': 1038, 'depth': 4, 'learning_rate': 0.014469868593285846, 'l2_leaf_reg': 6.6002572988548085, 'random_strength': 5.638468971075214, 'border_count': 180, 'bagging_temperature': 0.9952363164175864} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "D:\Skills\Kaggle\ml-regression_concrete-strength\venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Павел\AppData\Local\Temp\ipykernel_4892\2355813120.py", line 33, in objective
    scores = cross_val_score(
             ^^^^^^^^^^^^^^^^
  File "D:\Skills\Kaggle\ml-regression_concrete-strength\venv\Lib\site-packages\sklearn\utils\_param_validation.py", line 218, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "D:\Skills\Kaggle\ml-regression_concrete-strength\venv\Lib\site-packages\sk

KeyboardInterrupt: 